In [1]:
from tsnecuda import TSNE
import pandas as pd
import numpy as np
import pickle
import sys 
import os 
import torch 
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm_notebook

device =torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
try :
    torch.set_deterministic(True)
except:
    pass
torch.set_printoptions(precision=10)

/om/user/ehoseini/miniconda3/lib/python3.7/site-packages/torch/__init__.py:422: UserWarning: torch.set_deterministic is deprecated and will be removed in a future release. Please use torch.use_deterministic_algorithms instead
  "torch.set_deterministic is deprecated and will be removed in a future "


In [2]:
save_dir = '/mindhive/evlab/u/Shared/Greta_Eghbal_manifolds/extracted/'
model_identifier= 'NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed_data_figure_5_KNN.pkl'
d_fig_5=pickle.load(open(os.path.join(save_dir,model_identifier),'rb'))
x_bar=d_fig_5['x_bar']
data=d_fig_5['data']
accuracies=d_fig_5['accuracies']
epoch_bar=d_fig_5['epoch_bar']
epoch_batch=d_fig_5['epoch_batch']

# tSNE when its done on each epoch 

In [ ]:
layers = ["layer_1_Linear","layer_2_Linear", "layer_3_Linear"]
for idx,layer in tqdm_notebook(enumerate(layers)):
    fig, axes = plt.subplots(6, 10, figsize=(28*10/6,28),dpi=75,frameon=False)
    plt.suptitle(f'layer={layer}')
    for hier_idx in tqdm_notebook(np.arange(6)):        
        feat=torch.tensor(data['data'])
        labels=torch.tensor(np.ravel(np.stack(data['targets'])))
        all_epochs=np.ravel(data['epochs']).astype(int)
        num_colors=labels.unique().shape[0]
        h0=cm.get_cmap('inferno',num_colors+int(np.ceil(num_colors/10)))
        line_cols=np.flipud(h0(np.arange(num_colors)/num_colors))
        all_col=np.asarray([line_cols[x-1,:] for x in labels])
        for epoch in range(10):
            epoch_fix=epoch+1
            epoch_idx=np.where(all_epochs==epoch_fix)[0]
            feat_epoch=feat[epoch_idx,:]
            selections=np.random.choice(feat_epoch.shape[0],250,replace=False);
            feat_selection=feat_epoch[selections,:]
            ax=axes[5-hier_idx,epoch]
            X_embed_early = TSNE(n_components=2, perplexity=30, learning_rate=10).fit_transform(feat_selection)
            labels_early=labels[epoch_idx]
            label_selection=labels_early[selections]
            all_col_early=all_col[epoch_idx,:]
            all_col_selection=all_col_early[selections,:]
            ax.scatter(X_embed_early[:,0],X_embed_early[:,1],c=all_col_selection,s=25,alpha=.7)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.tick_params(
                    axis='x',          # changes apply to the x-axis
                    which='both',      # both major and minor ticks are affected
                    bottom=False,      # ticks along the bottom edge are off
                    top=False,         # ticks along the top edge are off
                    labelbottom=False) 
            ax.tick_params(
                    axis='y',          # changes apply to the x-axis
                    which='both',      # both major and minor ticks are affected
                    bottom=False,      # ticks along the bottom edge are off
                    top=False,         # ticks along the top edge are off
                    labelbottom=False) 
            ax.set_xlim((X_embed_early[:,0].min()+.1*X_embed_early[:,0].min(),X_embed_early[:,0].max()+.1*X_embed_early[:,0].max()))
            ax.set_ylim((X_embed_early[:,1].min()+.1*X_embed_early[:,1].min(),X_embed_early[:,1].max()+.1*X_embed_early[:,1].max()))
            ax.set_axis_off()
    plt.tight_layout() 